In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
import pandas as pd

In [31]:
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
}

In [19]:
def predict(model, test_image_name, device):
    transform = image_transforms['test']
    test_image = Image.open(test_image_name)
    test_image = test_image.convert('RGB')
#     plt.imshow(test_image)
    test_image_tensor = transform(test_image)
    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).to(device)
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).to(device)
    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
        return topclass

In [ ]:
model = models.resnet34(pretrained=True)
model.load_state_dict(torch.load('Models/qc_brightness_resnet.pth'))
model = model.to('cpu')

In [45]:
fucci_dir = 'Nuclei/2_all'
summary_dr = []
for i in os.listdir(fucci_dir):
    name = i.split('.')[0]
    row = [name]
    image_name = f'{fucci_dir}/{i}'
    output = predict(model, image_name, device)
    output = output.detach().cpu().numpy()[0][0]
    row = np.append(row, output)
    summary_dr.append(row)

In [41]:
tmp = pd.DataFrame(summary_dr, columns=['cell_name', 'type'])
tmp['type'] = tmp['type'].replace(mapper)

In [37]:
mapper = {'0': 'bad_bright', '1': 'bad_dark', '2': 'normal'}

In [43]:
tmp.type.value_counts()

normal        20348
bad_dark       5024
bad_bright     4403
Name: type, dtype: int64

In [44]:
tmp.to_csv('Dataframes/qc_brightness.csv', index=False)

In [23]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_data:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images).to(device)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 1000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 1000 test images: 98 %


In [25]:
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))
with torch.no_grad():
    for data in test_data:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(3):
    print('Accuracy of %5s : %2d %%' % (
        f'{i}', 100 * class_correct[i] / class_total[i]))

Accuracy of     0 : 96 %
Accuracy of     1 : 100 %
Accuracy of     2 : 98 %


In [26]:
classes

['bad_bright', 'bad_dark', 'normal']